In [1]:
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import matplotlib as mpl

In [2]:
CYC_COUNT = 10
mpl.rcParams["axes.prop_cycle"] = plt.cycler(
    "color", plt.cm.brg(np.linspace(0, 0.5, CYC_COUNT))
)

In [3]:
END = 'Z_0.095'

frames_file = Path(f'./photom_detections/photometry_candidates_ALLZ_COMBINE_REFITS{END}.pkl.gz')
frame_records: pd.DataFrame = pd.read_pickle(frames_file)

print(len(frame_records))
grouped_frame_records = frame_records.groupby(['tel', 'ut', 'camera'])

grouped_frame_records[['fit2_c0','fit2_c1','fit2_c2']].mean()


image_save = Path(f'./photom_detections/pngs_COMBINE{END}/')

173782


In [19]:
from astropy.modeling import models, fitting
from astropy.stats import sigma_clip

print('"tel","ut","camera","zp0_a","zp0_b","zp0_Z1"')

for grp, recs in grouped_frame_records:
    airmass = recs["airmass"]
    c0_obs = recs["fit2_c0"]
    c0_model = models.Polynomial1D(degree=1)
    c0_fitter = fitting.FittingWithOutlierRemoval(
        fitter=fitting.LinearLSQFitter(),
        outlier_func=sigma_clip,
        sigma=3,
        niter=5,
    )
    c0_model_fit, clipped_obs = c0_fitter(model=c0_model, x=airmass, y=c0_obs)

    csv_line = (
        [grp[0], grp[1], f'"{grp[2]}"']
        + list(c0_model_fit.parameters)
        + [sum(c0_model_fit.parameters)]
    )

    csv_line = [
        str(el) if not isinstance(el, float) else f"{el:.4f}" for el in csv_line
    ]

    print(",".join(csv_line))

"tel","ut","camera","zp0_a","zp0_b","zp0_Z1"
1,1,"ML5644917",21.5502,-0.0974,21.4528
1,2,"ML0430516",20.8780,-0.1081,20.7699
1,3,"ML0330316",21.3904,-0.1326,21.2578
1,4,"ML1252020",21.4996,-0.0863,21.4134
1,5,"ML0010316",21.4672,-0.1223,21.3449
1,5,"ML6054917",21.5391,-0.1640,21.3752
1,6,"ML0420516",21.4974,-0.1391,21.3583
1,6,"ML6304917",21.5942,-0.1103,21.4838
1,7,"ML9081020",21.7569,-0.1411,21.6158
1,8,"ML6094917",21.5309,-0.1211,21.4098
2,1,"ML1082020",21.1299,-0.0868,21.0430
2,2,"ML9071020",21.1747,-0.1641,21.0106
2,3,"ML1242020",21.5806,-0.1293,21.4513
2,4,"ML1282020",21.1628,-0.1205,21.0423
2,5,"ML1292020",21.4795,-0.1414,21.3381
2,6,"ML1352020",20.4013,-0.1103,20.2911
2,7,"ML9091020",21.6595,-0.1740,21.4854
2,8,"ML1262020",21.5205,-0.0513,21.4693
3,1,"ML9731020",22.2162,-0.3584,21.8578
3,2,"ML0601722",21.7203,-0.2015,21.5187
3,2,"ML9061020",22.0503,-0.2877,21.7627
3,2,"ML9191020",20.6005,-0.2380,20.3624
3,3,"ML9741020",22.2028,-0.2608,21.9419
3,4,"ML9101020",21.9252,-0.1678,21.

In [4]:
grp: tuple
recs: pd.DataFrame


columns = [
    "fit2_c0",
    "fit2_c1",
    "fit2_c2",
]
groups = [
    'c0/"ZP"',
    "c1",
    "c2",
]
invert = [
    False,
    False,
    False,
]
ylims = [
    # (frame_records["fit2_c0"].min() - 0.1, frame_records["fit2_c0"].max() + 0.1),
    # (frame_records["fit2_c1"].min() - 0.01, frame_records["fit2_c1"].max() + 0.01),
    # (frame_records["fit2_c2"].min() - 0.01, frame_records["fit2_c2"].max() + 0.01),
    (17.5, 22.5),
    (0.4, 0.6),
    (0.06, 0.13),
]
splitlen = int(len(columns) / len(groups))


config_dump = {}


for grp, recs in grouped_frame_records:
    print(grp, len(recs))

    if pd.isna(recs.iloc[0][columns[-1]]):
        continue

    fig: Figure
    axes: list[Axes]

    fig = plt.figure(figsize=(15, 15))
    axes = [
        plt.subplot(6, 6, (1, 8)),
        plt.subplot(6, 6, (3, 10)),
        plt.subplot(6, 6, (5, 12)),
        plt.subplot(6, 6, (13, 27)),
        plt.subplot(6, 6, (16, 24)),
        plt.subplot(6, 6, (28, 36)),
    ]

    def plotviolin(
        ax: Axes,
        dataset: pd.DataFrame,
        dataset_names,
        title,
        invert_y,
        ylims,
        #side="both",
    ):
        grouped_datasets = dataset.groupby("airmass_group")
        medians = []
        xs = []
        for airmass_group, grouprecs in grouped_datasets:
            medians.append(grouprecs[dataset_names].median())
            x = int(airmass_group) / 10 + 0.05
            xs.append(x)
            violinplot = ax.violinplot(
                grouprecs[dataset_names],
                positions=[x],
                widths=0.1,
                showmedians=True,
                #side=side,
            )
        mean_airmass = dataset["airmass"].mean()
        mean_value = dataset[dataset_names[0]].mean()
        # b, a = np.polyfit(xs, medians, deg=1)
        b, a = np.polyfit(dataset["airmass"], dataset[dataset_names], deg=1)
        # print(dataset["airmass"].values)
        # print(dataset[dataset_names[0]].values)

        corrcoef = np.corrcoef(
            dataset["airmass"].values, dataset[dataset_names[0]].values
        )
        # print(corrcoef)
        x = np.linspace(1, 2, 2)
        ax.plot(
            x,
            a + b * x,
            "--",
            c="k",
            alpha=0.4,
            label=f"a={a[0]:.3f}, b={b[0]:.3f}, corr={corrcoef[0,1]:.2f}",
        )
        ax.scatter(
            [mean_airmass],
            [mean_value],
            c="k",
            marker="X",
            label=f"x,y mean: {mean_airmass:.2f},{mean_value:.4f}",
        )
        ax.legend()

        ax.set_title(title)
        ax.set_ylim(ylims)
        ax.set_xlabel("airmass")
        ax.set_xticks(np.arange(1, 2.1, 0.1))
        if invert_y:
            ax.invert_yaxis()
        return violinplot

    def plotfit(
        ax: Axes,
        dataset: pd.DataFrame,
        norm_c0: bool,
        title: str,
    ):
        def _model(x, c0, c1, c2):
            if norm_c0:
                c0 = 0
            return x**2 * c2 + x * c1 + c0

        x = np.linspace(-0.6, 1.6, 200)

        grouped_datasets = dataset.groupby("airmass_group")

        for airmass_group, grouprecs in grouped_datasets:
            c0 = grouprecs["fit2_c0"].median()
            c1 = grouprecs["fit2_c1"].iloc[0]
            c2 = grouprecs["fit2_c2"].iloc[0]
            airmass = airmass_group / 10
            #print(airmass, c0, c1, c2)
            y = _model(x, c0, c1, c2)

            ax.plot(x, y, label=f"Z = {airmass}")

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.hlines(0, *xlim, colors="k")
        ax.vlines(0, *ylim, colors="k")
        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)
        if norm_c0:
            title += " (normalised $c_0=0$)"
        ax.set_title(title)
        ax.set_ylabel("response $g-\mathrm{{mag_{{inst}}}} / mag$")
        ax.set_xlabel("color $g-r$ / mag")

    def plotfitfit(
        ax: Axes,
        dataset: pd.DataFrame,
        norm_c0: bool,
        title: str,
    ):
        if not norm_c0:
            raise ValueError("func can only do normalised c0, `norm_c0`=False")

        axins = [
            ax.inset_axes([0.1, 0.6, 0.3, 0.3], xlim=(-0.61, -0.5), ylim=(-0.31, -0.2)),
            ax.inset_axes([0.5, 0.1, 0.4, 0.4], xlim=(1.41, 1.61), ylim=(0.9, 1.1)),
        ]
        for axin in axins:
            ax.indicate_inset_zoom(axin, edgecolor="black")

        def _fitmodel(x, c1, c2):
            return x**2 * c2 + x * c1

        x = np.linspace(-0.6, 1.6, 200)

        b_c1, a_c1 = np.polyfit(dataset["airmass"], dataset["fit2_c1"], deg=1)
        b_c2, a_c2 = np.polyfit(dataset["airmass"], dataset["fit2_c2"], deg=1)

        dp=4
        conf = [[round(a_c1,dp), round(b_c1,dp)], round(a_c2,dp)]

        print(conf)

        config_dump[':'.join(str(x) for x in grp)] = conf

        for airmass_group in range(10, 20):
            airmass = airmass_group / 10
            c1 = b_c1 * airmass + a_c1
            c2 = b_c2 * airmass + a_c2
            #print(airmass, c1, c2)
            y = _fitmodel(x, c1, c2)

            for axis in ax, *axins:
                axis.plot(x, y, label=f"Z = {airmass}")

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.hlines(0, *xlim, colors="k")
        ax.vlines(0, *ylim, colors="k")
        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)
        if norm_c0:
            title += " (normalised $c_0=0$)"
        ax.set_title(title)
        ax.set_ylabel("response $g-\mathrm{{mag_{{inst}}}} / mag$")
        ax.set_xlabel("color $g-r$ / mag")

    for i in range(len(groups)):
        cols = columns[i * splitlen : (i + 1) * splitlen]
        plotviolin(
            axes[i],
            dataset=recs,
            dataset_names=cols,
            title=groups[i],
            invert_y=invert[i],
            ylims=ylims[i],
        )

    plotfit(
        ax=axes[3],
        dataset=recs,
        norm_c0=False,
        title="fit lines",
    )
    plotfit(
        ax=axes[4],
        dataset=recs,
        norm_c0=True,
        title="fit lines",
    )
    plotfitfit(
        ax=axes[5],
        dataset=recs,
        norm_c0=True,
        title="fit line gradients",
    )

    axes[0].set_ylabel("mag")
    mainlabel = f"tel-{grp[0]}_ut-{grp[1]}_cam-{grp[2]}"
    fig.suptitle(f"{mainlabel}  ({len(recs)})")

    fig.tight_layout()
    
    figfile = image_save / f"{mainlabel}.png"
    print(figfile)
    plt.savefig(figfile)
    #plt.show()
    plt.close()

    # break

(1, 1, 'ML5644917') 4000


[[0.449, 0.0206], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-1_cam-ML5644917.png
(1, 2, 'ML0430516') 4000
[[0.5132, 0.0186], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-2_cam-ML0430516.png
(1, 3, 'ML0330316') 4000
[[0.4616, 0.0183], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-3_cam-ML0330316.png
(1, 4, 'ML1252020') 4000
[[0.4407, 0.0144], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-4_cam-ML1252020.png
(1, 5, 'ML0010316') 3978
[[0.4702, 0.0228], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-5_cam-ML0010316.png
(1, 5, 'ML6054917') 3946
[[0.4798, 0.0209], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-5_cam-ML6054917.png
(1, 6, 'ML0420516') 3726
[[0.4368, 0.0217], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-6_cam-ML0420516.png
(1, 6, 'ML6304917') 4000
[[0.4318, 0.0253], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-1_ut-6_cam-ML6304917.png
(1, 7, 'ML9081020') 4000
[[0.424, 0.0192], 0.095]
photom_detections/pngs_COMBINEZ_

/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[[0.4414, 0.0276], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-3_ut-1_cam-ML9731020.png
(3, 2, 'ML0601722') 3212


/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


...

[[0.4276, 0.0323], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-4_ut-6_cam-ML9061020.png
(4, 6, 'ML9871020') 3480
[[0.4234, 0.0328], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-4_ut-6_cam-ML9871020.png
(4, 7, 'ML0591722') 3600


/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[[0.4363, 0.0263], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-4_ut-7_cam-ML0591722.png
(4, 7, 'ML9191020') 3303
[[0.4533, 0.0391], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-4_ut-7_cam-ML9191020.png
(4, 8, 'ML0601722') 3600


/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/djarvis/src/goto-fpe/.venv/lib64/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[[0.4306, 0.0216], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-4_ut-8_cam-ML0601722.png
(4, 8, 'ML9121020') 3483
[[0.4376, 0.0214], 0.095]
photom_detections/pngs_COMBINEZ_0.095/tel-4_ut-8_cam-ML9121020.png


In [5]:
output = {
    grp+":L:g:r": conf for grp, conf in config_dump.items()
}

import json

print(json.dumps(output).replace(', "', ',\n"'))

{"1:1:ML5644917:L:g:r": [[0.449, 0.0206], 0.095],
"1:2:ML0430516:L:g:r": [[0.5132, 0.0186], 0.095],
"1:3:ML0330316:L:g:r": [[0.4616, 0.0183], 0.095],
"1:4:ML1252020:L:g:r": [[0.4407, 0.0144], 0.095],
"1:5:ML0010316:L:g:r": [[0.4702, 0.0228], 0.095],
"1:5:ML6054917:L:g:r": [[0.4798, 0.0209], 0.095],
"1:6:ML0420516:L:g:r": [[0.4368, 0.0217], 0.095],
"1:6:ML6304917:L:g:r": [[0.4318, 0.0253], 0.095],
"1:7:ML9081020:L:g:r": [[0.424, 0.0192], 0.095],
"1:8:ML6094917:L:g:r": [[0.4649, 0.0168], 0.095],
"2:1:ML1082020:L:g:r": [[0.4405, 0.0304], 0.095],
"2:2:ML9071020:L:g:r": [[0.4259, 0.018], 0.095],
"2:3:ML1242020:L:g:r": [[0.4497, 0.0166], 0.095],
"2:4:ML1282020:L:g:r": [[0.4782, 0.0142], 0.095],
"2:5:ML1292020:L:g:r": [[0.4282, 0.0195], 0.095],
"2:6:ML1352020:L:g:r": [[0.5694, 0.0109], 0.095],
"2:7:ML9091020:L:g:r": [[0.4299, 0.0191], 0.095],
"2:8:ML1262020:L:g:r": [[0.4393, 0.0143], 0.095],
"3:1:ML9731020:L:g:r": [[0.4414, 0.0276], 0.095],
"3:2:ML0601722:L:g:r": [[0.4195, 0.0304], 0.095],
"3